In [1]:
from grapher import plot_model_stats

### Isak Andersson - AI23 - Deep Learning

# Laboration

---

Behåll denna (gör till LaTex):
- Output size=((Input size−Kernel size+2×Padding​)/Stride)+1
- Den beskriver output size av conv layer. Hittade i översättningen till torch (dense == linear)

In [2]:
plot_model_stats([
    "./logs/keras/modelstats_08-12.csv",
    "./logs/keras/modelstats_09-12.csv",
    "./logs/keras/modelstats_10-12.csv",
    "./logs/keras/modelstats_11-12.csv",
    "./logs/keras/modelstats_12-12.csv"
    ])

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': ('<b>%{hovertext}</b><br><br>var' ... '}<br>value=%{y}<extra></extra>'),
              'hovertext': array([   20.,    41.,    57., ..., 15238., 15251., 15269.]),
              'legendgroup': 'running_reward',
              'line': {'color': '#636efa', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'running_reward',
              'showlegend': True,
              'type': 'scattergl',
              'x': array([   20,    41,    57, ..., 15238, 15251, 15269], dtype=int64),
              'xaxis': 'x',
              'y': array([151.25      , 147.68292683, 160.87719298, ..., 356.35      ,
                          366.9       , 359.45      ]),
              'yaxis': 'y'},
             {'hovertemplate': ('<b>%{hovertext}</b><br><br>var' ... '}<br>value=%{y}<extra></extra>'),
              'hovertext': array([   20.,    41.,    57., ..., 15238., 15251., 15269.]),
              'legendgroup': 'max_reward',
              'line': {'color': '#EF553B', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'max_reward',
              'showlegend': True,
              'type': 'scattergl',
              'x': array([   20,    41,    57, ..., 15238, 15251, 15269], dtype=int64),
              'xaxis': 'x',
              'y': array([385., 410., 500., ..., 865., 865., 865.]),
              'yaxis': 'y'}],
    'layout': {'legend': {'title': {'text': 'variable'}, 'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'Running Reward over time'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'Episode'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'value'}}}
})